#Group 1
Name: Salma Chaaban <br>
Student number: 301216551 <br>
Name: Hodan Ahmed Yusuf<br>
Student number: 301226634<br>
Name: Jonathan Au<br>
Student number: 300827701<br>
Name: Landon Essex<br>
Student number: 301349452<br>
Name: Michael Angelo Cabalinan <br>
Student number: 300924795 <br>
COMP 216 - Final project


#Overview
We will implement an end-to-end IoT solution that will satisfy your client needs (this is based on your previous labs). We will use the MQTT protocol for this implementation. There will be publisher clients  that sends data to a broker as well as subscriber clients that receives data from the broker. The broker sends data to the appropriate subscriber. <br>
You will design and build both types of clients according to specifications. The diagram shows an overly simplified architecture of the intended system.<br>
Each component is described more fully below along with the weight contribution towards the final grade. Each component must be implemented be well designed class.


##Summary rubric
The weight includes design, implementation and demonstration of that particular component.<br>
No written documentation is required, only your python code files. Your source files should have only one class per file.<br>
Item	Marks(%)<br>
Broker	5<br>
Publisher	40<br>
Subscriber	45<br>
Quality of code	5<br>
Video (demonstration)	5<br>
Total	100<br>


[link text](https://)# group_1_data_generator.py

In [ ]:
# from lab 6 but with minor changes

import math
import random

#1. Class
class DataGenerator:

    def __init__(self, ymin=18, ymax=25, daily_mean=19, daily_amps=[2, 1.5, 2, 1.8, 2.2], stddev=0.5, readings_per_day=100):
        self.ymin = ymin
        self.ymax = ymax
        self.readings_per_day = readings_per_day

        # Parameters for daily sine fluctuations
        self.daily_mean = daily_mean
        self.daily_amps = daily_amps
        self.set_daily_freqs()

        self.stddev = stddev

        self.t = 0

    @property
    def readings_per_day(self):
        return self._readings_per_day

    # setter for readings per day
    @readings_per_day.setter
    def readings_per_day(self, value):
        if value <= 0:
            raise ValueError('readings per day must be greater than 0')
        self._readings_per_day = value
        self.set_daily_freqs()

    def set_daily_freqs(self):
        self.daily_freqs = [2 * math.pi / self._readings_per_day, 0.5 * math.pi / self._readings_per_day,
                            1.8 * math.pi / self._readings_per_day, 0.8 * math.pi / self._readings_per_day,
                            1.1 * math.pi / self._readings_per_day]  # 100 temperature readings per day

    #2. private method
    def __rand_values(self):
        return random.uniform(0,1)

    def _normal_random(self):
        return random.gauss(0, self.stddev)

    @property
    def value(self):
        # Daily sin fluctuation
        daily_pattern = self.daily_mean

        # Other fluctuations
        for i in range(len(self.daily_amps)):
            daily_pattern += self.daily_amps[i] * math.sin(self.daily_freqs[i] * self.t)

        # noise
        noise = self._normal_random()

        y = daily_pattern + noise

        # scale y to be in the range of 0 - 1
        y = y / (sum(self.daily_amps) + self.daily_mean)

        # scale y to be in the range ymin - ymax
        yrange = self.ymax - self.ymin
        y = self.ymin + yrange * y

        # increase time
        self.t += 1

        return y


#group_1_broker.py
You will install the Eclipse mosquito broker and ensure that it is working as expected.<br>
# New Section

In [ ]:
import paho.mqtt.client as mqtt

class Broker:
    def __init__(self, broker_host, broker_port):
        self.client = mqtt.Client()
        self.broker_host = broker_host
        self.broker_port = broker_port

    def connect(self):
        self.client.connect(self.broker_host, self.broker_port)
        self.client.loop_start()

    def publish(self, topic, message):
        self.client.publish(topic, message)

    def subscribe(self, topic, callback):
        self.client.subscribe(topic)
        self.client.on_message = callback

    def disconnect(self):
        self.client.loop_stop()
        self.client.disconnect()

In [ ]:
import time
from broker import Broker

def message_received(client, userdata, message):
    print(f"Received message '{str(message.payload.decode('utf-8'))}' on topic '{message.topic}'")

#create instance of Broker
broker = Broker("localhost", 1883)

#connect to MQTT Broker
broker.connect()

#subscribe to topic
broker.subscribe("sensor/temperature", message_received)

#publish messages
broker.publish("sensor/temperature", "25.5°C")

#allow some time for communication
time.sleep(2)

#disconnect from broker
broker.disconnect()

#group_1_publisher.py
The publisher will generate data to send to the broker at regular intervals. The data value must be random with a pattern (I know that this is a contradiction). Think of the value of a particular stock on the stock exchange or the outdoor temperature around your home. <br>
This must be implemented as gui and must include an interface to change the various parameters of this publisher.<br>
You will configure and run multiple publisher clients to simulate multiple devices.
###Publisher – Value generation
This must be implemented in a class in a separate file. (Just import the filename without the py extension in the file where you want to use the logic). This must be based on Week_09_lab_08_simulator.docx. You must not limit the number of values generated.<br><br>
The design should be such that it is easy to use and flexible enough to be able to give the data value in the required pattern.<br><br>
[Specifications of lab 8 more rigidly enforced.]
###Publisher – Packaging the above values
The above value must be tagged with at least a time stamp packet id and packaged as a json object before transmission. You decide what other features you need to encapsulate in your package.[See lab 13 util.py]<br>
###Publisher – Sending data to broker
•	You will send the above packaged data to the broker under an agreed topic. [See lab13 publisher.py]<br>
•	You must miss transmission with a frequency of about 1 in very 100 transmissions. This must not be deterministic!<br>
###Publisher – Extras
These extras are for bonus points and will only be considered if all the normal specifications are satisfied adequately.<br>
•	To simulate a real-world scenario, occasionally skip blocks of transmissions (or sets of transmission). This condition must not throw the subscriber into confusion.<br>
•	Transmit “wild data” something that is completely off the chart. Again your subscriber should be able to handle this.<br>
•	Anything that will add value to your project. You must make me aware of these.


In [ ]:
import tkinter as tk
from tkinter import ttk
import random
import threading
import time
import json
import paho.mqtt.client as mqtt
import group_1_data_generator as dg

# Global variables for initial values
DEFAULT_MIN_VALUE = '0'
DEFAULT_MAX_VALUE = '45'
DEFAULT_DAILY_MEAN = '20'
DEFAULT_READINGS_PER_DAY = '48'
MQTT_BROKER = 'localhost'
DEFAULT_MQTT_TOPIC = 'Temperature Over Time'
TRANSMISSION_ERROR_RATE = 0.01

class TemperaturePublisherGui:
    def __init__(self, root):
        self.root = root
        self.root.title('Publisher')
        self.is_publishing = False  # Initialize the publishing flag

        # set weight to make columns and rows resizable
        self.root.columnconfigure(0, weight=1)
        self.root.columnconfigure(1, weight=1)
        self.root.rowconfigure(0, weight=1)
        self.root.rowconfigure(1, weight=1)
        self.root.rowconfigure(2, weight=1)
        self.root.rowconfigure(3, weight=1)
        self.root.rowconfigure(4, weight=1)
        self.root.rowconfigure(5, weight=1)

        # Default values for the entry widgets
        self.min_value_var = tk.DoubleVar(value=DEFAULT_MIN_VALUE)
        self.max_value_var = tk.DoubleVar(value=DEFAULT_MAX_VALUE)
        self.daily_mean_var = tk.DoubleVar(value=DEFAULT_DAILY_MEAN)
        self.readings_per_day_var = tk.DoubleVar(value=DEFAULT_READINGS_PER_DAY)

        self.topic_var = tk.StringVar(value=DEFAULT_MQTT_TOPIC)

        # add labels and entries
        self.min_label = ttk.Label(root, text='Min Value:')
        self.min_entry = ttk.Entry(root, textvariable=self.min_value_var)
        self.min_label.grid(row=1, column=0, padx=10, sticky='e')
        self.min_entry.grid(row=1, column=1, padx=10, sticky='w')

        self.max_label = ttk.Label(root, text='Max Value:')
        self.max_entry = ttk.Entry(root, textvariable=self.max_value_var)
        self.max_label.grid(row=2, column=0, padx=10, sticky='e')
        self.max_entry.grid(row=2, column=1, padx=10, sticky='w')

        self.daily_mean_label = ttk.Label(root, text='Daily Mean:')
        self.daily_mean_entry = ttk.Entry(root, textvariable=self.daily_mean_var)
        self.daily_mean_label.grid(row=3, column=0, padx=10, sticky='e')
        self.daily_mean_entry.grid(row=3, column=1, padx=10, sticky='w')

        self.readings_per_day_label = ttk.Label(root, text='Readings Per Day:')
        self.readings_per_day_entry = ttk.Entry(root, textvariable=self.readings_per_day_var)
        self.readings_per_day_label.grid(row=4, column=0, padx=10, sticky='e')
        self.readings_per_day_entry.grid(row=4, column=1, padx=10, sticky='w')

        # update button
        self.update_button = ttk.Button(root, text="Update", command=self.update_values)
        self.update_button.grid(row=5, column=0, columnspan=2, pady=10)
        self.update_button['state'] = 'disabled'

        # add start and stop buttons
        self.start_button = ttk.Button(root, text='Start publishing', command=self.start_publishing)
        self.stop_button = ttk.Button(root, text='Stop publishing', command=self.stop_publishing)
        self.start_button.grid(row=6, column=0, padx=10, pady=10)
        self.stop_button.grid(row=6, column=1, padx=10, pady=10)

        # entry value for Topic
        self.topic_label = ttk.Label(root, text='Topic:')
        self.topic_entry = ttk.Entry(root, textvariable=self.topic_var)
        self.topic_label.grid(row=7, column=0, padx=10, pady=10, sticky='e')
        self.topic_entry.grid(row=7, column=0, padx=10, pady=10, sticky='w')

        # set start and stop buttons
        self.start_button['state'] = 'disabled'
        self.stop_button['state'] = 'disabled'

        # add connect and disconnect buttons
        self.connect_button = ttk.Button(root, text='Connect', command=self.connect_to_broker)
        self.disconnect_button = ttk.Button(root, text='Disconnect', command=self.disconnect_from_broker)
        self.connect_button.grid(row=0, column=0, padx=10, pady=10)
        self.disconnect_button.grid(row=0, column=1, padx=10, pady=10)

        # set connect and disconnect buttons
        self.connect_button['state'] = 'enabled'
        self.disconnect_button['state'] = 'disabled'

        # Bonus part
        self.separator = ttk.Separator(root, orient='horizontal')
        self.separator.grid(row=8, column=0, columnspan=2, sticky='ew', padx=10, pady=10)
        self.separator_label = ttk.Label(root, text='Bonus')
        self.separator_label.grid(row=9, column=0, columnspan=2, sticky='ns', padx=10, pady=10)

        # add button to transmit wild data
        self.wild_data_button = ttk.Button(root, text='Transmit Wild Data', command=self.wild_data)
        self.wild_data_button.grid(row=10, column=0, padx=10, pady=10)

        # add button to skip block of data
        self.skip_button = ttk.Button(root, text='Skip Block of Data', command=self.skip_blocks)
        self.skip_button.grid(row=10, column=1, padx=10, pady=10)

        # set the buttons
        self.wild_data_button['state'] = 'disabled'
        self.skip_button['state'] = 'disabled'

        # setup MQTT client
        self.mqtt_client = mqtt.Client(mqtt.CallbackAPIVersion.VERSION1)
        self.mqtt_client.on_connect = self.on_connect
        self.mqtt_client.on_disconnect = self.on_disconnect

        # add trace to detect value changes to the min, max, mean
        self.min_value_var.trace_add('write', self.on_value_changed)
        self.max_value_var.trace_add('write', self.on_value_changed)
        self.daily_mean_var.trace_add('write', self.on_value_changed)
        self.readings_per_day_var.trace_add('write', self.on_value_changed)

        self.data_generator = dg.DataGenerator(ymin=self.min_value_var.get(),
                                               ymax=self.max_value_var.get(),
                                               daily_mean=self.daily_mean_var.get(),
                                               readings_per_day=self.readings_per_day_var.get())

        # initialize packet counter
        self.packet_id_counter = 0
        # initialize skip next count for simulation of lost packets
        self.skip_next_count = 0

        # initialize flag for wild data
        self.is_wild_data = False

    def wild_data(self):
        self.is_wild_data = True

    def skip_blocks(self):
        self.skip_next_count = random.randint(1, 10)

    def on_connect(self, client, userdata, flags, rc):
        if rc == 0:
            print(f'Connected to "{MQTT_BROKER}"')
            self.connect_button['state'] = 'disabled'
            self.disconnect_button['state'] = 'normal'
            # enable start button
            self.start_button['state'] = 'normal'
        else:
            print(f'Connection failed, with result code:  {rc}')

    def on_disconnect(self, client, userdata, rc):
        if rc != 0:
            print(f'Unexpected disconnection. Result code: {rc}')
        else:
            print(f'Disconnected')

        # set flag to stop the loop
        self.loop_active = False
        self.connect_button['state'] = 'normal'
        self.disconnect_button['state'] = 'disabled'

        # disabled buttons
        self.start_button['state'] = 'disabled'
        self.stop_button['state'] = 'disabled'
        self.wild_data_button['state'] = 'disabled'
        self.skip_button['state'] = 'disabled'

        # stop publishing
        self.stop_publishing()

    def connect_to_broker(self):
        self.mqtt_client.connect(MQTT_BROKER, 1883, 60)
        self.mqtt_client.loop_start()

        # update button states
        self.connect_button['state'] = 'disabled'
        self.disconnect_button['state'] = 'normal'

    def disconnect_from_broker(self):
        self.stop_publishing()
        self.mqtt_client.loop_stop()

        # update button states
        self.connect_button['state'] = 'normal'
        self.disconnect_button['state'] = 'disabled'

    def start_publishing(self):
        if not self.is_publishing:
            self.is_publishing = True
            self.temperature_thread = threading.Thread(target=self.publish_temperature)
            self.temperature_thread.start()
            self.stop_button['state'] = 'normal'
            self.start_button['state'] = 'disabled'
            self.topic_entry['state'] = 'disabled'
            self.wild_data_button['state'] = 'normal'
            self.skip_button['state'] = 'normal'

    def stop_publishing(self):
        if self.is_publishing:
            self.is_publishing = False
            self.temperature_thread.join()
            self.stop_button['state'] = 'disabled'
            self.start_button['state'] = 'normal'
            self.topic_entry['state'] = 'normal'
            self.wild_data_button['state'] = 'disabled'
            self.skip_button['state'] = 'disabled'

    def publish_temperature(self):
        while self.is_publishing:
            self.packet_id_counter += 1

            # generate the data with packet and timestamp
            temperature_value = self.data_generator.value

            if self.skip_next_count > 0:
                self.skip_next_count -= 1
                print(f'Skipping packet {self.packet_id_counter}')
                continue

            if self.is_wild_data:

                if random.random() < 0.5:
                    temperature_value += 100000000
                else:
                    temperature_value += -100000000
                self.is_wild_data = False

            data = {
                "temperature": temperature_value,
                "timestamp": time.strftime('%Y-%m-%d %H:%M:%S'),
                "packet_id": self.packet_id_counter
            }

            # convert data to json string
            json_data = json.dumps(data)

            # randomly skip 1% of the packets
            if random.random() < TRANSMISSION_ERROR_RATE:
                print(f'Skipping packet {self.packet_id_counter}')
                continue

            # publish json data
            print(f'publishing {self.topic_var.get()}: {json_data}')
            self.mqtt_client.publish(self.topic_var.get(), payload=json_data)

            time.sleep(1) # sleep 1 second between publishing

    def update_values(self):
        self.data_generator.ymin = self.min_value_var.get()
        self.data_generator.ymax = self.max_value_var.get()
        self.data_generator.daily_mean = self.daily_mean_var.get()
        self.data_generator.readings_per_day = self.readings_per_day_var.get()
        self.update_button['state'] = 'disabled'

    # called if values change
    def on_value_changed(self, *args):
        # enable the update button when a value changes
        self.update_button['state'] = 'normal'


if __name__ == '__main__':

    def on_closing():
        if app.is_publishing():
            app.stop_publishing()

        # disconnect
        app.disconnect_from_broker()
        root.destroy()

    root = tk.Tk()
    app = TemperaturePublisherGui(root)
    root.protocol('WM_DELETE_WINDOW', on_closing)

    root.mainloop()

#group_1_subscriber.py
The subscriber accept data from the broker and process it. It will decode the data and decide how to process it.<br>
This is best implemented as a GUI application.<br>
You will configure and run multiple subscriber clients to simulate multiple devices.<br>
###Subscriber – Receiving data from the broker
•	You will listen to messages from the broker under an agreed topic.<br>
•	You will decode the message and decide how to handle the data<br>
###Subscriber – Handling the data/absence of data
This section is important, because this is where you imbue your personality on this project. You have to decide what is out of range data. You also have to be able to detect missing transmission. <br>
•	Handle sensible data. Display data both in text and visual formats.<br>
•	Detecting and handling out of range (erroneous) data. <br>
•	Detecting and handling missing data.



In [ ]:
#lab 11 minor changes
import tkinter as tk
from tkinter import ttk
import paho.mqtt.client as mqtt
from group_1_dynamic_chart import DynamicChart
import json
import threading

# Global variables
MQTT_BROKER = 'localhost'
DEFAULT_MQTT_TOPIC = 'Temperature Over Time'

class TemperatureSubscriberGUI:
    def __init__(self, root):
        self.root = root
        self.root.title('Subscriber')
        self.inDisconnect = False
        self.next_packet_id = 0
        self.is_publishing = False

        # Set weight to make columns and rows resizable
        for i in range(6):
            self.root.rowconfigure(i, weight=1)

        # add a valid data range min and max values entry boxes
        self.valid_min_var = tk.StringVar(value=0)
        self.valid_min_label = ttk.Label(root, text='Valid Min Value:')
        self.valid_min_entry = ttk.Entry(root, textvariable=self.valid_min_var)
        self.valid_min_label.grid(row=1, column=0, padx=10, sticky='ew')
        self.valid_min_entry.grid(row=1, column=1, padx=10, sticky='ew')

        self.valid_max_var = tk.StringVar(value=45)
        self.valid_max_label = ttk.Label(root, text='Valid Max Value:')
        self.valid_max_entry = ttk.Entry(root, textvariable=self.valid_max_var)
        self.valid_max_label.grid(row=2, column=0, padx=10, sticky='ew')
        self.valid_max_entry.grid(row=2, column=1, padx=10, sticky='ew')

        self.text_label = ttk.Label(root, text='Recieved Temperature Values:')
        self.text_label.grid(row=3, column=0, columnspan=2, padx=10, pady=10, sticky='ns')

        self.text_display = tk.Text(root, wrap='word', width=40, height=10)
        self.text_display.grid(row=4, column=0, columnspan=2, padx=10, pady=10, sticky='nsew')

        scrollbar = tk.Scrollbar(root, command=self.text_display.yview)
        scrollbar.grid(row=4, column=3, sticky='nsew')
        self.text_display.config(yscrollcommand=scrollbar.set)

        # add connect and disconnect buttons
        self.connect_button = ttk.Button(root, text='Connect', command=self.connect_to_broker)
        self.disconnect_button = ttk.Button(root, text='Disconnect', command=self.disconnect_from_broker)
        self.connect_button.grid(row=0, column=0, padx=10, pady=10)
        self.disconnect_button.grid(row=0, column=1, padx=10, pady=10)

        self.connect_button['state'] = 'enabled'
        self.disconnect_button['state'] = 'disabled'

        # add start and stop buttons
        self.start_button = ttk.Button(root, text='Start publishing', command=self.start_publishing)
        self.stop_button = ttk.Button(root, text='Stop publishing', command=self.stop_publishing)
        self.start_button.grid(row=5, column=0, padx=10, pady=10)
        self.stop_button.grid(row=5, column=1, padx=10, pady=10)

        self.stop_button['state'] = 'disabled'

        # entry value for topic
        self.topic_var = tk.StringVar(value=DEFAULT_MQTT_TOPIC)
        self.topic_label = ttk.Label(root, text='Topic:')
        self.topic_entry = ttk.Entry(root, textvariable=self.topic_var)
        self.topic_label.grid(row=6, column=0, padx=10, pady=10, sticky='e')
        self.topic_entry.grid(row=6, column=1, padx=10, pady=10, sticky='w')

        # MQTT client setup
        self.mqtt_client = mqtt.Client(mqtt.CallbackAPIVersion.VERSION1)
        self.mqtt_client.on_message = self.on_message
        self.mqtt_client.on_connect = self.on_connect
        self.mqtt_client.on_disconnect = self.on_disconnect

        # Create a canvas widget
        self.chart_canvas = tk.Canvas(root)
        self.chart_canvas.grid(row=0, column=5, rowspan=6, padx=10, pady=10, sticky='ew')

        # Pass the canvas widget to DynamicChart
        initial_data = [0] * 20
        self.dynamic_chart = DynamicChart(initial_data, self.chart_canvas)


    def connect_to_broker(self):
        self.mqtt_client.connect(MQTT_BROKER, 1883, 60)
        try:
            self.mqtt_client.loop_start()
            print('Started loop')
        except Exception as e:
            print('Unable to start loop')
            print(e)
        # update button states
        self.connect_button['state'] = 'disabled'
        self.disconnect_button['state'] = 'normal'

    def disconnect_from_broker(self):
        self.inDisconnect = True
        self.mqtt_client.loop_stop()
        self.mqtt_client.disconnect()
        # update button states
        self.connect_button['state'] = 'normal'
        self.disconnect_button['state'] = 'disabled'

    def on_connect(self, client, userdata, flags, rc):
        if rc == 0:
            self.dynamic_chart.set_title(self.topic_var.get())
            print(f'Connected to "{MQTT_BROKER}"')
            self.mqtt_client.subscribe(self.topic_var.get())
            self.connect_button['state'] = 'disabled'
            self.disconnect_button['state'] = 'normal'
            # enable start button
            self.start_button['state'] = 'normal'
        else:
            print(f'Connection failed, with result code:  {rc}')

    def on_disconnect(self, client, userdata, rc):
        if rc != 0:
            print(f'Unexpected disconnection. Result code: {rc}')
        else:
            print(f'Disconnected')

        self.connect_button['state'] = 'normal'
        self.disconnect_button['state'] = 'disabled'
        self.topic_entry['state'] = 'normal'

    def start_publishing(self):
        if not self.is_publishing:
            self.is_publishing = True
            self.temperature_thread = threading.Thread(target=self.publish_temperature)
            self.temperature_thread.start()
            self.stop_button['state'] = 'normal'
            self.start_button['state'] = 'disabled'
            self.topic_entry['state'] = 'disabled'

    def stop_publishing(self):
        if self.is_publishing:
            self.is_publishing = False
            self.temperature_thread.join()
            self.stop_button['state'] = 'disabled'
            self.start_button['state'] = 'normal'
            self.topic_entry['state'] = 'normal'

    def on_message(self, client, userdata, msg):
        if self.inDisconnect:
            return

        # Decode the message and load it as json
        json_data = json.loads(msg.payload.decode('utf-8'))
        temperature_value = json_data['temperature']
        timestamp = json_data['timestamp']
        packet_id = json_data['packet_id']

        if (self.next_packet_id > 0 and packet_id != self.next_packet_id):
            print(f"Warning: received packet {packet_id} out of order")
            self.text_display.insert(tk.END, f'Packet {self.next_packet_id}')

        self.next_packet_id = packet_id + 1

        # try catch
        try:
            if (temperature_value < float(self.valid_min_var.get()) or temperature_value > float(self.valid_max_var.get())):
                print(f"Error: Value out of range, packet: {packet_id} received out of order")
                self.text_display.insert(tk.END, f'Packet: {packet_id} out of range')
                self.text_display.see(tk.END)
                return
        except ValueError:
            print(f'Invalid imput value')

        print(f"Received packet {packet_id} at {timestamp}: {self.topic_var.get()}")
        self.text_display.insert(tk.END, f'Packet {packet_id}: {temperature_value}')
        self.text_display.see(tk.END)

        # update chart
        self.dynamic_chart.add_value(temperature_value)

if __name__ == '__main__':
    root = tk.Tk()
    app = TemperatureSubscriberGUI(root)

    def on_closing():
        app.mqtt_client.disconnect()
        root.destroy()

    root.protocol("WM_DELETE_WINDOW", on_closing)
    root.mainloop()


##Quality of Code
These 5 points are to lose. You automatically start with full points and as the instructor notices code aberration these 5 points will evaporate. Code aberration will include design flaws and implementation blunders.
##Video of Project
You make a 5min video that will demonstrate each of the item in the rubric table. If an item is not clearly demonstrated, there might be a possibility that you might not get a point for that item.<br>
You need not demonstrate quality of code.
##Submission
1.	You will bundle all your files (except the video) into a single zip file (not wrar or anything else). The name of the file will be group_«your_group_number».zip e.g. group_1.zip.
2.	Your publisher code files must be called group_«your_group_number»_publisher.py e.g. group_1_publisher.py.
3.	Your subscriber code files must be called group_«your_group_number»_subscriber.py e.g. group_1_subscriber.py.
4.	All your other code files must be prefixed by you group number e.g. group_1_data_generator.py
5.	Must be uploaded to course dropbox.
6.	This is due at the end of week 13 (If you are going to the bonus marks, then the due date at the start of week 13).
7.	The video file must be uploaded directly in the drop box following the same naming conventions.


Dynamic Chart

In [ ]:
import tkinter as tk
import numpy as np

class DynamicChart:
    def __init__(self, initial_data, canvas):
        self.canvas = canvas
        self.values = initial_data
        self.t = 0
        self.init_chart()

    def init_chart(self):
        self.canvas.delete("all")
        self.canvas.config(width=500, height=250)
        self.draw_line()

    def add_value(self, new_value):
        self.values.pop(0)
        self.values.append(new_value)
        self.draw_line()

    def draw_line(self):
        self.canvas.delete("line")
        num_values = len(self.values)

        x_start = 40
        y_start = 220
        x_step = 410 / (num_values - 1)
        y_scale = 4

        smoothed_values = self.smooth_data(self.values)  # Smoothing the data

        for i in range(num_values - 1):
            x1 = x_start + i * x_step
            x2 = x_start + (i + 1) * x_step
            y1 = y_start - smoothed_values[i] * y_scale
            y2 = y_start - smoothed_values[i + 1] * y_scale
            self.canvas.create_line(x1, y1, x2, y2, fill="red", width=2, tags="line")

    def smooth_data(self, data, window_size=5):
        smoothed_data = np.convolve(data, np.ones(window_size) / window_size, mode='valid')
        return np.concatenate(([data[0]] * (len(data) - len(smoothed_data)), smoothed_data))

if __name__ == "__main__":
    root = tk.Tk()
    canvas = tk.Canvas(root)
    canvas.pack()

    initial_data = [0] * 20
    group_1_dynamic_chart = DynamicChart(initial_data, canvas)

    def update_chart():
        new_value = np.random.randint(0, 60)
        group_1_dynamic_chart.add_value(new_value)
        root.after(500, update_chart)

    update_chart()
    root.mainloop()
